# Exploratory Data Analysis (EDA)

## Цель

Исследовать исторические данные рекламных размещений в Telegram, выявить зависимости между CPM, CHANNEL_NAME, DATE и VIEWS, а также обосновать выбор модели и признаков.

---

## Импорты и загрузка данных

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

df = pd.read_csv("data/data_train.csv")
df.columns = df.columns.str.strip()
df["DATE"] = pd.to_datetime(df["DATE"])

df.head()
```

---

## Общая информация о данных

```python
df.info()
```

```python
df.describe()
```

**Комментарий:** данные не содержат пропусков в ключевых признаках. CPM и VIEWS имеют широкий диапазон значений.

---

## Распределение CPM и VIEWS

```python
sns.histplot(df["CPM"], bins=50, kde=True)
plt.title("Distribution of CPM")
plt.show()
```

```python
sns.histplot(df["VIEWS"], bins=50, kde=True)
plt.title("Distribution of VIEWS")
plt.show()
```

**Вывод:** распределения скошены вправо — используется log-преобразование.

---

## Зависимость CPM → VIEWS

```python
sample = df.sample(5000, random_state=42)

sns.scatterplot(data=sample, x="CPM", y="VIEWS", alpha=0.4)
plt.title("CPM vs VIEWS")
plt.show()
```

```python
sns.scatterplot(data=sample, x="CPM", y=np.log1p(sample["VIEWS"]), alpha=0.4)
plt.title("CPM vs log(VIEWS)")
plt.show()
```

---

## Анализ каналов

```python
top_channels = (
    df.groupby("CHANNEL_NAME")["VIEWS"]
      .mean()
      .sort_values(ascending=False)
      .head(10)
)

top_channels.plot(kind="bar")
plt.title("Top-10 channels by average VIEWS")
plt.show()
```

---

## Временные паттерны

```python
df["dayofweek"] = df["DATE"].dt.dayofweek
df["month"] = df["DATE"].dt.month
```

```python
sns.boxplot(data=df, x="dayofweek", y="VIEWS")
plt.title("VIEWS by day of week")
plt.show()
```

```python
sns.boxplot(data=df, x="month", y="VIEWS")
plt.title("VIEWS by month")
plt.show()
```

---

## Итоговые выводы

* Нелинейная зависимость CPM → VIEWS
* Существенные различия между каналами
* Наличие сезонности

На основании EDA выбрана модель CatBoostRegressor с категориальным признаком CHANNEL_NAME и log-трансформацией целевой переменной.